In [2]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
import tensorflow as tf

In [3]:
df = pd.read_csv('Churn_Modelling.csv')

In [4]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [5]:
df.nunique()

RowNumber          10000
CustomerId         10000
Surname             2932
CreditScore          460
Geography              3
Gender                 2
Age                   70
Tenure                11
Balance             6382
NumOfProducts          4
HasCrCard              2
IsActiveMember         2
EstimatedSalary     9999
Exited                 2
dtype: int64

In [6]:
X = df.iloc[:, 3:-1].values
y = df.iloc[:, -1].values
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [7]:
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
X

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

In [8]:
from sklearn.compose import ColumnTransformer
                                # tipo      #coluna    #preserva as outras colunas
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')

X = np.array(ct.fit_transform(X)) # Transformar em um matriz, a transformação vai primeira coluna

In [9]:
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=0)

In [11]:
pd.DataFrame(X_train).head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.0,0.0,1.0,667,0,34,5,0.0,2,1,0,163830.64
1,0.0,1.0,0.0,427,1,42,1,75681.52,1,1,1,57098.0
2,1.0,0.0,0.0,535,0,29,2,112367.34,1,1,0,185630.76
3,0.0,0.0,1.0,654,1,40,5,105683.63,1,1,0,173617.09
4,0.0,0.0,1.0,850,0,57,8,126776.3,2,1,1,132298.49


In [12]:
sc = StandardScaler()
X_train[:,[3, 5, 6, 7, 8, 11]] = sc.fit_transform(X_train[:,[3, 5, 6, 7, 8, 11]])
X_test[:,[3, 5, 6, 7, 8, 11]] = sc.transform(X_test[:,[3, 5, 6, 7, 8, 11]])

In [13]:
pd.DataFrame(X_train).head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.0,0.0,1.0,0.169582,0,-0.464608,0.006661,-1.215717,0.809503,1,0,1.106432
1,0.0,1.0,0.0,-2.304559,1,0.301026,-1.37744,-0.006312,-0.921591,1,1,-0.748664
2,1.0,0.0,0.0,-1.191196,0,-0.943129,-1.031415,0.579935,-0.921591,1,0,1.485335
3,0.0,0.0,1.0,0.035566,1,0.109617,0.006661,0.473128,-0.921591,1,0,1.276528
4,0.0,0.0,1.0,2.056114,0,1.736588,1.044737,0.810193,0.809503,1,1,0.558378


In [14]:
pd.DataFrame(X_test).head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.0,1.0,0.0,-0.552043,0,-0.368904,1.044737,0.879303,-0.921591,1,1,1.610857
1,1.0,0.0,0.0,-1.314903,0,0.109617,-1.031415,0.429722,-0.921591,1,0,0.49587
2,0.0,0.0,1.0,0.57163,0,0.301026,1.044737,0.308583,-0.921591,1,1,-0.424787
3,1.0,0.0,0.0,1.416961,1,-0.656016,-0.339364,0.575336,-0.921591,0,0,-0.187777
4,0.0,1.0,0.0,0.57163,1,-0.081791,0.006661,1.389611,0.809503,1,1,0.616842


In [30]:
# Criando um modelo sequencial
ann = tf.keras.models.Sequential()

In [34]:
# Adicionando a primeira camada oculta
# - 'units=6': 6 neurônios na camada
# - 'activation="relu"': Função de ativação ReLU para introduzir não-linearidade
# Esta camada terá como entrada o número de features (dimensão das variáveis independentes)
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [ ]:
# Adicionando a camada de saída
# - 'units=1': Um único neurônio porque o problema é de classificação binária
# - 'activation="sigmoid"': Retorna um valor entre 0 e 1 (probabilidade)
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))